In [31]:
%%capture
!pip install -q dagshub mlflow langchain langchain-openai langchain-together langchain-community

In [2]:
#@markdown **You need to sign up for [DagsHub](https://dagshub.com/user/sign_up) , then enter the name of the repository you'd like to create, and your username and email.**

#@markdown Enter the repository name for the project:
REPO_NAME= "komrag" #@param {type:"string"}

#@markdown Enter the username of your DAGsHub account:
USER_NAME = "OsipovStas" #@param {type:"string"}

#@markdown Enter the email for your DAGsHub account:
EMAIL = "stasstels@gmail.com" #@param {type:"string"}

#@markdown ---

import IPython

In [ ]:
!dagshub login

import dagshub
TOKEN = dagshub.auth.get_token()

In [4]:
from google.colab import userdata
DH_TOKEN = userdata.get('LLM_MATH_COMP_DH_TOKEN')


In [5]:
import mlflow
import dagshub
import os
#DH_TOKEN = os.environ['LLM_MATH_COMP_DH_TOKEN']
username = f'{USER_NAME}'  # Replace with your DagsHub username
repository = f'{REPO_NAME}'  # Replace with your repository name
token = f'{DH_TOKEN}'  # Replace with your DagsHub token
os.environ['MLFLOW_TRACKING_USERNAME'] = username
os.environ['MLFLOW_TRACKING_PASSWORD'] = token
dagshub.auth.add_app_token(token)
dagshub.init(repo_name=REPO_NAME, repo_owner=USER_NAME)
mlflow.set_tracking_uri(f"https://dagshub.com/{USER_NAME}/{REPO_NAME}.mlflow")

Initialized MLflow to track repo "OsipovStas/komrag"

Repository OsipovStas/komrag initialized!

In [6]:
os.environ['LANGCHAIN_API_KEY'] = userdata.get("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_PROJECT'] = "komrag"

In [ ]:
from dagshub.notebook import save_notebook

save_notebook(repo=f"{USER_NAME}/{REPO_NAME}", path="./notebooks/exploratory.ipynb")

In [5]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("OsipovStas/komrag")

# Upload file
s3.upload_file(
    Bucket="komrag",  # name of the repo
    Filename="./life_begin.pdf",  # local path of file to upload
    Key="life_begin.pdf",  # remote path where to upload the file
)

Client created. Use the name of the repo (komrag) as the name of the bucket

In [6]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("OsipovStas/komrag")


s3.download_file(
    Bucket="komrag",  # name of the repo
    Key="life_begin.pdf",  #  remote path from where to download the file
    Filename="life_begin_copy.pdf",  # local path where to download the file
)

Client created. Use the name of the repo (komrag) as the name of the bucket

# LLM Configuration

In [9]:
import os

os.environ['AZURE_OPENAI_ENDPOINT'] = userdata.get("AZURE_OPENAI_ENDPOINT")
os.environ['AZURE_OPENAI_API_KEY'] = userdata.get('AZURE_KEY')
os.environ['OPENAI_API_VERSION'] = '2024-02-01'

In [12]:
model="gpt-4o-mini-2024-07-18"

In [19]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(model=model, temperature=0)

In [26]:
from langchain_together import ChatTogether

# choose from our 50+ models here: https://docs.together.ai/docs/inference-models
chat = ChatTogether(
    together_api_key=userdata.get("TOGETHER_KEY"),
    model="meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo",
)

In [32]:
chat.invoke("Hello, do you know russian? Write a response in it")

AIMessage(content='Да, я знаю русский язык. Как я могу помочь?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 22, 'total_tokens': 39, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e27be930-7dd4-41b5-8e65-f4b48ff07bf2-0', usage_metadata={'input_tokens': 22, 'output_tokens': 17, 'total_tokens': 39, 'input_token_details': {}, 'output_token_details': {}})